In [3]:
lenFragment = 1000
genomeFragmentList = []
fileName = "GCF_000013425.1_ASM1342v1_genomic.fna"#"Hodgkinia_cicadicola_tetund1_genome.fasta"#"Staph_argenteus_58113_genome.fasta""Deinococcus_radiodurans_R1dM1_genome.fasta"#
with open(fileName) as file:
    fragment = ""
    print(file.readline())
    for line in file.readlines():
        line = line.rstrip().upper()
        #print(line)
        if len(fragment) < lenFragment:
            pass
        else:
            genomeFragmentList.append(fragment[:lenFragment])
            fragment = fragment[lenFragment:]
        
        try:
            if line[0] == '>':
                print(line)
                break
            else:
                fragment += line
        except IndexError:
            print(line)
print(len(genomeFragmentList))

>NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome

2821


In [4]:
exampleSequence = 'AGGGCTTGCTATAT'
tetramerDicRepository = {key:[] for key in ['AAAA', 'AAAC', 'AAAG', 'AAAT', 'AACA', 'AACC', 'AACG', 'AACT', 'AAGA', 'AAGC', 'AAGG', 'AAGT', 'AATA', 'AATC', 'AATG', 'AATT', 'ACAA', 'ACAC', 'ACAG', 'ACAT', 'ACCA', 'ACCC', 'ACCG', 'ACCT', 'ACGA', 'ACGC', 'ACGG', 'ACGT', 'ACTA', 'ACTC', 'ACTG', 'ACTT', 'AGAA', 'AGAC', 'AGAG', 'AGAT', 'AGCA', 'AGCC', 'AGCG', 'AGCT', 'AGGA', 'AGGC', 'AGGG', 'AGGT', 'AGTA', 'AGTC', 'AGTG', 'AGTT', 'ATAA', 'ATAC', 'ATAG', 'ATAT', 'ATCA', 'ATCC', 'ATCG', 'ATCT', 'ATGA', 'ATGC', 'ATGG', 'ATGT', 'ATTA', 'ATTC', 'ATTG', 'ATTT', 'CAAA', 'CAAC', 'CAAG', 'CAAT', 'CACA', 'CACC', 'CACG', 'CACT', 'CAGA', 'CAGC', 'CAGG', 'CAGT', 'CATA', 'CATC', 'CATG', 'CATT', 'CCAA', 'CCAC', 'CCAG', 'CCAT', 'CCCA', 'CCCC', 'CCCG', 'CCCT', 'CCGA', 'CCGC', 'CCGG', 'CCGT', 'CCTA', 'CCTC', 'CCTG', 'CCTT', 'CGAA', 'CGAC', 'CGAG', 'CGAT', 'CGCA', 'CGCC', 'CGCG', 'CGCT', 'CGGA', 'CGGC', 'CGGG', 'CGGT', 'CGTA', 'CGTC', 'CGTG', 'CGTT', 'CTAA', 'CTAC', 'CTAG', 'CTAT', 'CTCA', 'CTCC', 'CTCG', 'CTCT', 'CTGA', 'CTGC', 'CTGG', 'CTGT', 'CTTA', 'CTTC', 'CTTG', 'CTTT', 'GAAA', 'GAAC', 'GAAG', 'GAAT', 'GACA', 'GACC', 'GACG', 'GACT', 'GAGA', 'GAGC', 'GAGG', 'GAGT', 'GATA', 'GATC', 'GATG', 'GATT', 'GCAA', 'GCAC', 'GCAG', 'GCAT', 'GCCA', 'GCCC', 'GCCG', 'GCCT', 'GCGA', 'GCGC', 'GCGG', 'GCGT', 'GCTA', 'GCTC', 'GCTG', 'GCTT', 'GGAA', 'GGAC', 'GGAG', 'GGAT', 'GGCA', 'GGCC', 'GGCG', 'GGCT', 'GGGA', 'GGGC', 'GGGG', 'GGGT', 'GGTA', 'GGTC', 'GGTG', 'GGTT', 'GTAA', 'GTAC', 'GTAG', 'GTAT', 'GTCA', 'GTCC', 'GTCG', 'GTCT', 'GTGA', 'GTGC', 'GTGG', 'GTGT', 'GTTA', 'GTTC', 'GTTG', 'GTTT', 'TAAA', 'TAAC', 'TAAG', 'TAAT', 'TACA', 'TACC', 'TACG', 'TACT', 'TAGA', 'TAGC', 'TAGG', 'TAGT', 'TATA', 'TATC', 'TATG', 'TATT', 'TCAA', 'TCAC', 'TCAG', 'TCAT', 'TCCA', 'TCCC', 'TCCG', 'TCCT', 'TCGA', 'TCGC', 'TCGG', 'TCGT', 'TCTA', 'TCTC', 'TCTG', 'TCTT', 'TGAA', 'TGAC', 'TGAG', 'TGAT', 'TGCA', 'TGCC', 'TGCG', 'TGCT', 'TGGA', 'TGGC', 'TGGG', 'TGGT', 'TGTA', 'TGTC', 'TGTG', 'TGTT', 'TTAA', 'TTAC', 'TTAG', 'TTAT', 'TTCA', 'TTCC', 'TTCG', 'TTCT', 'TTGA', 'TTGC', 'TTGG', 'TTGT', 'TTTA', 'TTTC', 'TTTG', 'TTTT']}
tetramerDic = {key:0 for key in ['AAAA', 'AAAC', 'AAAG', 'AAAT', 'AACA', 'AACC', 'AACG', 'AACT', 'AAGA', 'AAGC', 'AAGG', 'AAGT', 'AATA', 'AATC', 'AATG', 'AATT', 'ACAA', 'ACAC', 'ACAG', 'ACAT', 'ACCA', 'ACCC', 'ACCG', 'ACCT', 'ACGA', 'ACGC', 'ACGG', 'ACGT', 'ACTA', 'ACTC', 'ACTG', 'ACTT', 'AGAA', 'AGAC', 'AGAG', 'AGAT', 'AGCA', 'AGCC', 'AGCG', 'AGCT', 'AGGA', 'AGGC', 'AGGG', 'AGGT', 'AGTA', 'AGTC', 'AGTG', 'AGTT', 'ATAA', 'ATAC', 'ATAG', 'ATAT', 'ATCA', 'ATCC', 'ATCG', 'ATCT', 'ATGA', 'ATGC', 'ATGG', 'ATGT', 'ATTA', 'ATTC', 'ATTG', 'ATTT', 'CAAA', 'CAAC', 'CAAG', 'CAAT', 'CACA', 'CACC', 'CACG', 'CACT', 'CAGA', 'CAGC', 'CAGG', 'CAGT', 'CATA', 'CATC', 'CATG', 'CATT', 'CCAA', 'CCAC', 'CCAG', 'CCAT', 'CCCA', 'CCCC', 'CCCG', 'CCCT', 'CCGA', 'CCGC', 'CCGG', 'CCGT', 'CCTA', 'CCTC', 'CCTG', 'CCTT', 'CGAA', 'CGAC', 'CGAG', 'CGAT', 'CGCA', 'CGCC', 'CGCG', 'CGCT', 'CGGA', 'CGGC', 'CGGG', 'CGGT', 'CGTA', 'CGTC', 'CGTG', 'CGTT', 'CTAA', 'CTAC', 'CTAG', 'CTAT', 'CTCA', 'CTCC', 'CTCG', 'CTCT', 'CTGA', 'CTGC', 'CTGG', 'CTGT', 'CTTA', 'CTTC', 'CTTG', 'CTTT', 'GAAA', 'GAAC', 'GAAG', 'GAAT', 'GACA', 'GACC', 'GACG', 'GACT', 'GAGA', 'GAGC', 'GAGG', 'GAGT', 'GATA', 'GATC', 'GATG', 'GATT', 'GCAA', 'GCAC', 'GCAG', 'GCAT', 'GCCA', 'GCCC', 'GCCG', 'GCCT', 'GCGA', 'GCGC', 'GCGG', 'GCGT', 'GCTA', 'GCTC', 'GCTG', 'GCTT', 'GGAA', 'GGAC', 'GGAG', 'GGAT', 'GGCA', 'GGCC', 'GGCG', 'GGCT', 'GGGA', 'GGGC', 'GGGG', 'GGGT', 'GGTA', 'GGTC', 'GGTG', 'GGTT', 'GTAA', 'GTAC', 'GTAG', 'GTAT', 'GTCA', 'GTCC', 'GTCG', 'GTCT', 'GTGA', 'GTGC', 'GTGG', 'GTGT', 'GTTA', 'GTTC', 'GTTG', 'GTTT', 'TAAA', 'TAAC', 'TAAG', 'TAAT', 'TACA', 'TACC', 'TACG', 'TACT', 'TAGA', 'TAGC', 'TAGG', 'TAGT', 'TATA', 'TATC', 'TATG', 'TATT', 'TCAA', 'TCAC', 'TCAG', 'TCAT', 'TCCA', 'TCCC', 'TCCG', 'TCCT', 'TCGA', 'TCGC', 'TCGG', 'TCGT', 'TCTA', 'TCTC', 'TCTG', 'TCTT', 'TGAA', 'TGAC', 'TGAG', 'TGAT', 'TGCA', 'TGCC', 'TGCG', 'TGCT', 'TGGA', 'TGGC', 'TGGG', 'TGGT', 'TGTA', 'TGTC', 'TGTG', 'TGTT', 'TTAA', 'TTAC', 'TTAG', 'TTAT', 'TTCA', 'TTCC', 'TTCG', 'TTCT', 'TTGA', 'TTGC', 'TTGG', 'TTGT', 'TTTA', 'TTTC', 'TTTG', 'TTTT']}
import time

In [5]:
begin = time.time()
print('hello1')
for n, fragment in enumerate(genomeFragmentList):
    tetramerDic = {key:0 for key in ['AAAA', 'AAAC', 'AAAG', 'AAAT', 'AACA', 'AACC', 'AACG', 'AACT', 'AAGA', 'AAGC', 'AAGG', 'AAGT', 'AATA', 'AATC', 'AATG', 'AATT', 'ACAA', 'ACAC', 'ACAG', 'ACAT', 'ACCA', 'ACCC', 'ACCG', 'ACCT', 'ACGA', 'ACGC', 'ACGG', 'ACGT', 'ACTA', 'ACTC', 'ACTG', 'ACTT', 'AGAA', 'AGAC', 'AGAG', 'AGAT', 'AGCA', 'AGCC', 'AGCG', 'AGCT', 'AGGA', 'AGGC', 'AGGG', 'AGGT', 'AGTA', 'AGTC', 'AGTG', 'AGTT', 'ATAA', 'ATAC', 'ATAG', 'ATAT', 'ATCA', 'ATCC', 'ATCG', 'ATCT', 'ATGA', 'ATGC', 'ATGG', 'ATGT', 'ATTA', 'ATTC', 'ATTG', 'ATTT', 'CAAA', 'CAAC', 'CAAG', 'CAAT', 'CACA', 'CACC', 'CACG', 'CACT', 'CAGA', 'CAGC', 'CAGG', 'CAGT', 'CATA', 'CATC', 'CATG', 'CATT', 'CCAA', 'CCAC', 'CCAG', 'CCAT', 'CCCA', 'CCCC', 'CCCG', 'CCCT', 'CCGA', 'CCGC', 'CCGG', 'CCGT', 'CCTA', 'CCTC', 'CCTG', 'CCTT', 'CGAA', 'CGAC', 'CGAG', 'CGAT', 'CGCA', 'CGCC', 'CGCG', 'CGCT', 'CGGA', 'CGGC', 'CGGG', 'CGGT', 'CGTA', 'CGTC', 'CGTG', 'CGTT', 'CTAA', 'CTAC', 'CTAG', 'CTAT', 'CTCA', 'CTCC', 'CTCG', 'CTCT', 'CTGA', 'CTGC', 'CTGG', 'CTGT', 'CTTA', 'CTTC', 'CTTG', 'CTTT', 'GAAA', 'GAAC', 'GAAG', 'GAAT', 'GACA', 'GACC', 'GACG', 'GACT', 'GAGA', 'GAGC', 'GAGG', 'GAGT', 'GATA', 'GATC', 'GATG', 'GATT', 'GCAA', 'GCAC', 'GCAG', 'GCAT', 'GCCA', 'GCCC', 'GCCG', 'GCCT', 'GCGA', 'GCGC', 'GCGG', 'GCGT', 'GCTA', 'GCTC', 'GCTG', 'GCTT', 'GGAA', 'GGAC', 'GGAG', 'GGAT', 'GGCA', 'GGCC', 'GGCG', 'GGCT', 'GGGA', 'GGGC', 'GGGG', 'GGGT', 'GGTA', 'GGTC', 'GGTG', 'GGTT', 'GTAA', 'GTAC', 'GTAG', 'GTAT', 'GTCA', 'GTCC', 'GTCG', 'GTCT', 'GTGA', 'GTGC', 'GTGG', 'GTGT', 'GTTA', 'GTTC', 'GTTG', 'GTTT', 'TAAA', 'TAAC', 'TAAG', 'TAAT', 'TACA', 'TACC', 'TACG', 'TACT', 'TAGA', 'TAGC', 'TAGG', 'TAGT', 'TATA', 'TATC', 'TATG', 'TATT', 'TCAA', 'TCAC', 'TCAG', 'TCAT', 'TCCA', 'TCCC', 'TCCG', 'TCCT', 'TCGA', 'TCGC', 'TCGG', 'TCGT', 'TCTA', 'TCTC', 'TCTG', 'TCTT', 'TGAA', 'TGAC', 'TGAG', 'TGAT', 'TGCA', 'TGCC', 'TGCG', 'TGCT', 'TGGA', 'TGGC', 'TGGG', 'TGGT', 'TGTA', 'TGTC', 'TGTG', 'TGTT', 'TTAA', 'TTAC', 'TTAG', 'TTAT', 'TTCA', 'TTCC', 'TTCG', 'TTCT', 'TTGA', 'TTGC', 'TTGG', 'TTGT', 'TTTA', 'TTTC', 'TTTG', 'TTTT']}
    #print('hello')
    for pos in range(0,len(fragment)-3):
        tetramer = fragment[pos:pos+4]  
        #print('hello2')
        try:
            tetramerDic[tetramer] += 1
        except KeyError:
            pass

    for key, value in tetramerDic.items():
        tetramerDicRepository[key].append(value)

    if n%50 == 0:
        end = time.time()
        print(f'Till {n} run took {end-begin}')
        begin = time.time()
print('done')

hello1
Till 0 run took 0.0009889602661132812
Till 50 run took 0.02007150650024414
Till 100 run took 0.018460988998413086
Till 150 run took 0.01922440528869629
Till 200 run took 0.021865129470825195
Till 250 run took 0.018770933151245117
Till 300 run took 0.016283512115478516
Till 350 run took 0.016536235809326172
Till 400 run took 0.017090559005737305
Till 450 run took 0.017283916473388672
Till 500 run took 0.01741337776184082
Till 550 run took 0.017190933227539062
Till 600 run took 0.016340017318725586
Till 650 run took 0.01646709442138672
Till 700 run took 0.01441049575805664
Till 750 run took 0.014292716979980469
Till 800 run took 0.014873266220092773
Till 850 run took 0.014170646667480469
Till 900 run took 0.012868404388427734
Till 950 run took 0.014400959014892578
Till 1000 run took 0.012796163558959961
Till 1050 run took 0.01405787467956543
Till 1100 run took 0.014779329299926758
Till 1150 run took 0.01327061653137207
Till 1200 run took 0.014587640762329102
Till 1250 run took 0.0

In [6]:
print(tetramerDicRepository['AAAC'])

[6, 9, 10, 9, 6, 6, 10, 6, 7, 6, 6, 4, 4, 5, 8, 5, 3, 6, 5, 5, 8, 6, 10, 5, 6, 8, 7, 7, 9, 9, 4, 14, 14, 10, 7, 6, 8, 4, 9, 7, 4, 8, 8, 1, 7, 11, 5, 11, 3, 5, 5, 8, 8, 12, 7, 9, 6, 9, 6, 6, 8, 11, 5, 5, 8, 5, 7, 5, 6, 0, 7, 2, 5, 0, 5, 7, 8, 5, 5, 7, 6, 5, 4, 6, 6, 5, 7, 7, 6, 5, 7, 2, 8, 9, 8, 7, 3, 0, 5, 7, 6, 3, 7, 3, 4, 7, 7, 7, 6, 3, 6, 7, 2, 7, 12, 2, 6, 13, 6, 10, 6, 7, 7, 5, 7, 9, 6, 10, 7, 3, 4, 5, 6, 7, 7, 6, 9, 10, 4, 5, 6, 7, 10, 4, 5, 8, 7, 2, 5, 5, 5, 5, 4, 8, 4, 3, 8, 9, 4, 4, 2, 3, 7, 6, 5, 3, 5, 6, 6, 6, 5, 5, 2, 4, 8, 6, 6, 11, 10, 4, 4, 0, 0, 6, 4, 4, 7, 5, 4, 4, 3, 8, 4, 11, 3, 1, 7, 6, 3, 3, 5, 5, 3, 7, 4, 6, 10, 12, 9, 4, 10, 3, 15, 20, 9, 1, 4, 1, 4, 5, 5, 2, 8, 6, 4, 4, 4, 7, 7, 7, 14, 6, 4, 8, 3, 6, 6, 5, 2, 4, 4, 8, 7, 5, 7, 11, 3, 7, 11, 6, 7, 5, 4, 9, 6, 11, 4, 3, 5, 7, 2, 4, 5, 10, 9, 10, 6, 7, 5, 3, 11, 4, 2, 4, 5, 2, 13, 12, 9, 4, 10, 6, 6, 7, 6, 7, 7, 2, 10, 5, 6, 4, 3, 5, 4, 13, 9, 8, 8, 8, 6, 6, 3, 5, 10, 8, 7, 7, 7, 3, 5, 5, 8, 10, 8, 8, 6, 2, 7, 2, 6

In [7]:
# on command line write
#     pip install pandas

# Write this and give it your dictionary (hope the shape is as expected)
import pandas as pd
tetramerDic = tetramerDicRepository
# should be shaped like
#     {'AAAA':[num,num,num...], 
#      'AAAT':[num,num,num...], ... }

df = pd.DataFrame(data=tetramerDic)
#df.to_csv('give_to_matthew_plz.csv',index=False)

In [8]:
df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,16,6,12,14,10,3,0,9,10,6,...,1,2,6,1,5,7,17,5,8,11
1,26,9,9,18,7,2,2,5,19,5,...,3,3,5,5,4,4,12,6,5,11
2,18,10,7,16,10,2,2,9,12,1,...,1,6,3,4,2,3,13,6,3,7
3,32,9,9,27,6,2,6,3,9,2,...,1,9,10,2,5,4,13,4,4,8
4,16,6,15,15,5,1,8,4,9,4,...,2,4,7,2,4,5,12,1,7,4


In [9]:
sums = df.sum()
df_freq = df.loc[:,'AAAA':'TTTT'].div(sums, axis=1)

In [10]:
sums

AAAA    42172
AAAC    17603
AAAG    21019
AAAT    40782
AACA    20823
        ...  
TTGT    21416
TTTA    39462
TTTC    20901
TTTG    21800
TTTT    42829
Length: 256, dtype: int64

In [11]:
df_freq.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,0.000379,0.000341,0.000571,0.000343,0.000480,0.000366,0.000000,0.000609,0.000567,0.000547,...,0.00013,0.000110,0.000265,0.000075,0.000457,0.000327,0.000431,0.000239,0.000367,0.000257
1,0.000617,0.000511,0.000428,0.000441,0.000336,0.000244,0.000215,0.000338,0.001078,0.000456,...,0.00039,0.000166,0.000221,0.000377,0.000366,0.000187,0.000304,0.000287,0.000229,0.000257
2,0.000427,0.000568,0.000333,0.000392,0.000480,0.000244,0.000215,0.000609,0.000681,0.000091,...,0.00013,0.000331,0.000132,0.000301,0.000183,0.000140,0.000329,0.000287,0.000138,0.000163
3,0.000759,0.000511,0.000428,0.000662,0.000288,0.000244,0.000644,0.000203,0.000511,0.000182,...,0.00013,0.000497,0.000442,0.000151,0.000457,0.000187,0.000329,0.000191,0.000183,0.000187
4,0.000379,0.000341,0.000714,0.000368,0.000240,0.000122,0.000859,0.000271,0.000511,0.000365,...,0.00026,0.000221,0.000309,0.000151,0.000366,0.000233,0.000304,0.000048,0.000321,0.000093


In [12]:
df_freq.to_csv('tetraFrequency.csv',index=False)